In [1]:
import numpy as np

TRACK_FILE = "snapped_tracks_lj_ve"
DATA_DIR = "../.datasets/gps_tracks/resampled/"

tracks = np.load(DATA_DIR + TRACK_FILE + ".npy")

In [2]:
distances = []
for track in tracks:
    i = 1
    t0 = track[0]
    t1 = track[i]
    while(i < track.shape[0]-1):
        dist = np.sqrt(np.power(t0[0]-t1[0], 2) + np.power(t0[1]-t1[1], 2))
        #print(dist)
        i += 1
        t0 = t1
        t1 = track[i]
        distances.append(dist)

distances = np.array(distances)
print("mean: \t", np.mean(distances), 
      "\nmedian:\t", np.median(distances), 
      "\nmax:\t",np.amax(distances), 
      "\nmin:\t", np.amin(distances))

mean: 	 0.00411600823988 
median:	 0.00344765281732 
max:	 4.375018881 
min:	 0.0


In [20]:
NEW_DISTANCE = 0.0001
def resample_track(track):
    print(track.shape)
    new_track = []
    
    i = 1
    
    t0 = track[0]
    t1 = track[i]
    
    new_track.append(t0)
    
    while(i < track.shape[0]-1):
        if i % 20 == 0:
            print(i)
        
        tempX1 = tempX2 = D = a = b = c = k = n = 0
        new_append_point = 0
        #print(i, t0, t1)
        
        ## compute the d2 distance between the two gps points:
        dist = np.sqrt(np.power(t0[0]-t1[0], 2) + np.power(t0[1]-t1[1], 2))
        
        ## depending on the distance, the different cases to handle:
        
        if dist == NEW_DISTANCE:
            
            ## distance is just right: append the seccond point -t1 to the new track
            
            i += 1
            new_append_point = [t1[0], t1[1]]
            new_track.append(new_append_point)
            print("Perfect point - append")
            t0 = t1
            t1 = track[i]

        elif dist > NEW_DISTANCE:
            """
            ## calculated distance is smaller: 
            ## calculate a new point on the line between t0 and t1 at distance NEW_DISTANCE from t0 
            ## and append it to the new track
            """
            if not t0[0] == t1[0]: 
                ## if the x values of both points are not the same we use "this" calculation to compute the new point
                
                ## basic linear equation (y = kx + n) that connects points t0 and t1
                k = (t1[1] - t0[1])/(t1[0] - t0[0])
                n = t0[1] - k * t0[0]
                
                """
                ## the solution to the equation d^2 = (x - x')^2 + (y - y')^2,
                ## where (x,y)=t0 and (x',y') is the new point at distance NEW_DISTANCE from 
                ## point t0 on the linear equation connecting t0 and t1
                """
                
                a = 1 + np.power(k, 2)
                b = 2 * (n * k - t0[0] - t0[1] * k)
                c = np.power(n,2) + np.power(t0[0],2) + np.power(t0[1],2) - 2*n*t0[1] - np.power(NEW_DISTANCE,2)
                D = np.power(b,2) - 4 * a * c
                if D < 0:
                    print("Error D value!",
                             "\ntempX1: ", tempX1, 
                             "\ntempX2: ", tempX2,
                             "\nT0x: ", t0[0],
                             "\nT0y: ", t0[1],
                             "\nT1x: ", t1[0],
                             "\nT1y: ", t1[1],
                             "\ni:",i,
                             "\nD:", D)
                    break
                elif D == 0:
                    print("D value problem!",
                             "\ntempX1: ", tempX1, 
                             "\ntempX2: ", tempX2,
                             "\nT0x: ", t0[0],
                             "\nT0y: ", t0[1],
                             "\nT1x: ", t1[0],
                             "\nT1y: ", t1[1],
                             "\ni:",i,
                             "\nD:", D)
                    break
                else:
                    tempX1 = round((-b + np.sqrt(D)) / (2 * a), 10)
                    tempX2 = round((-b - np.sqrt(D)) / (2 * a), 10)

                if t0[0] < t1[0]:
                    if t0[0] <= tempX1 and tempX1 <= t1[0]:
                        t0[0] = tempX1

                    elif t0[0] < tempX2 and tempX2 < t1[0]:
                        t0[0] = tempX2

                    else:
                        print("Error with new X value!",
                             "\ntempX1: ", tempX1, 
                             "\ntempX2: ", tempX2,
                             "\nT0x: ", t0[0],
                             "\nT0y: ", t0[1],
                             "\nT1x: ", t1[0],
                             "\nT1y: ", t1[1],
                             "\ni:",i)
                        break
                else:
                    if t1[0] <= tempX1 and tempX1 <= t0[0]:
                        t0[0] = tempX1

                    elif t1[0] < tempX2 and tempX2 < t0[0]:
                        t0[0] = tempX2

                    else:
                        print("Error with new X value!",
                             "\ntempX1: ", tempX1, 
                             "\ntempX2: ", tempX2,
                             "\nT0x: ", t0[0],
                             "\nT0y: ", t0[1],
                             "\nT1x: ", t1[0],
                             "\nT1y: ", t1[1],
                             "\ni:",i)
                        break

                t0[1] = t0[0] * k + n
            else:
                ## if the x values of both points are the same, we just need to change the y value 
                ## of the first point in the correct direction + or -, depending on the y value of the second point
                
                if t0[1] < t1[1]:
                    t0[1] = t0[1] + NEW_DISTANCE
                
                elif t1[1] <t0[1]:
                    t0[1] = t0[1] - NEW_DISTANCE
                
                else:
                    ## isolated case where t0 == t1:
                    ## we just need to select the next t1 point in the track and continue with the while loop
                    
                    i += 1
                    #t0 = t1
                    t1 = track[i]
                    continue
            
            ## append the calculated point to the new track
            new_append_point = [t0[0], t0[1]]
            new_track.append(new_append_point)
            #print("append")
            
        else:
            i += 1
            #t0 = t1
            t1 = track[i]
    
    return np.array(new_track)
print(tracks[1].shape)
new_track = resample_track(tracks[1])
print("new_track:", new_track.shape)

(499, 2)
(499, 2)
D value problem! 
tempX1:  0 
tempX2:  0 
T0x:  45.655371213 
T0y:  12.207327504 
T1x:  45.6554069519 
T1y:  12.2113046646 
i: 16 
D: 0.0
new_track: (137, 2)
